Prepare Dataset
* How does it handle invalid values
* What do we want to do with null values
* Do we want to summarise, group or filter the data?

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("source/Diabetes.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


The nulls are showing up as 0s, big problem for this kind of data!

* If there are nulls and you want to replace nulls:
    
        df.fillna(0) --> every null is replaced with 0

* If you want to drop rows with nulls:

        df.dropna(0) --> you can choose how many nulls need to exist in row to drop it


In [4]:
df.fillna(0)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
df = df.fillna(0)

To choose which columns to work with, extract them into a new dataframe

In [6]:
df2 = df[["Glucose", "BMI", "Age", "Outcome"]]

df2.head()

,Glucose,BMI,Age,Outcome
0,148,33.6,50,1
1,85,26.6,31,0
2,183,23.3,32,1
3,89,28.1,21,0
4,137,43.1,33,1


In [7]:
df2.describe()

,Glucose,BMI,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000
mean,120.894531,31.992578,33.240885,0.348958
std,31.972618,7.884160,11.760232,0.476951
min,0.000000,0.000000,21.000000,0.000000
25%,99.000000,27.300000,24.000000,0.000000
50%,117.000000,32.000000,29.000000,0.000000
75%,140.250000,36.600000,41.000000,1.000000
max,199.000000,67.100000,81.000000,1.000000


Above we see the mins are 0 for some columns that it wouldn't make sense. These are problem zeros.

To get rid of 0s:

remove last column because 0s are valid there:
    
    df2.columns[:-1]

check for 0s as true or false
       
    df2[df2.columns[:-1]] == 0
    
check if any column has a zero on the horizontal axis

    (df2[df2.coumns[:-1]] == 0).any(axis=1)
    
using a boolean mask so use loc 
use the tilde to invert trues to falses to keep all the falses


In [15]:
df3 = df2.loc[~(df2[df2.columns[:-1]] == 0).any(axis=1)]
df3.describe()
df3.info() #we lost about 15 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 767
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Glucose  752 non-null    int64  
 1   BMI      752 non-null    float64
 2   Age      752 non-null    int64  
 3   Outcome  752 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 29.4 KB


Grouping and arranging data

In [19]:
df3.groupby("Outcome") #returns an object
df3.groupby("Outcome").mean() #returns data by grouping

,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.876434,31.309426
1,142.488636,35.371970,37.015152


Choose which aggregate you want per column 

In [25]:
df3.groupby("Outcome").agg({"Glucose": "mean", "BMI": "median", "Age": "sum"})

,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.10,15279
1,142.488636,34.25,9772


Or, show aggregates for each column

In [30]:
df3.groupby("Outcome").agg(["mean", "median"])

Glucose               BMI               Age       
               mean median       mean median       mean median
Outcome                                                       
0        110.825820  107.5  30.876434  30.10  31.309426     27
1        142.488636  140.5  35.371970  34.25  37.015152     36

To split the dataframe into seperate dataframes for each group

In [31]:
pos = df3.loc[df3["Outcome"] == 1]
neg = df3.loc[df3["Outcome"] == 0]

print(pos.shape, neg.shape)

(264, 4) (488, 4)


Saving into files

In [32]:
df3.to_csv("source/clean_diabetes.csv", index=False)